In [5]:
import pandas as pd

file_path = 'Inventory_DataSet.xlsx'
data = pd.ExcelFile(file_path)
product_data = data.parse('Product')
user_data = data.parse('User')
print("Sheets in file:", data.sheet_names)
print("Product Data Preview:")
print(product_data.head())
print("User Data Preview:")
print(user_data.head())

Sheets in file: ['Product', 'User']
Product Data Preview:
                    ProductName  \
0                Cisco ISR 1101   
1                     HP 5406zl   
2        DOCSIS 3.1 Cable Modem   
3          Cellular Duplexer Rx   
4  SBB100 Splitter Trough Block   

                                         Description Product Image  \
0            ISR 1101 4 Ports GE Ethernet WAN Router           NaN   
1                          HP ProCurve Switch 5406zl           NaN   
2  Superfast speeds up to 10 gigabits per second ...     modem.jpg   
3     A multiplexer product that is RoHS6 compliant            NaN   
4   Splitter block  with copper-aluminium monopie...  splitter.jpg   

  Product Cateogy Name Model Number                         Serial Number  \
0               Router       XYZ123  6a38028c-6a71-4f44-b8f6-253bc7086a0a   
1               Switch       ABC456  7edc108e-45e0-4997-bc59-17852d16b689   
2                Modem       LMN789  e41e2e16-2945-4c0c-a584-48935742fe94   
3 

In [6]:
print("Missing values in Product data:")
print(product_data.isnull().sum())
print("Missing values in User data:")
print(user_data.isnull().sum())

Missing values in Product data:
ProductName               0
Description               0
Product Image           380
Product Cateogy Name      0
Model Number              0
Serial Number             0
StockLevel                0
ReorderPoint              0
Supplier Name             0
Supplier Mail             0
Supplier  Contact         0
Order Date                0
Quantity                  0
Order Status              0
dtype: int64
Missing values in User data:
UserId        0
UserName      0
Password      0
First Name    0
Last Name     0
Type          0
Phone         0
Email         0
dtype: int64


In [9]:
# Fill missing values in Product Image column
product_data['Product Image'] = product_data['Product Image'].fillna('No Image Available')


In [11]:
# Check and remove duplicates
product_data = product_data.drop_duplicates()
user_data = user_data.drop_duplicates()

print("Product data shape after removing duplicates:", product_data.shape)
print("User data shape after removing duplicates:", user_data.shape)


Product data shape after removing duplicates: (500, 14)
User data shape after removing duplicates: (3, 8)


In [13]:
product_data['Order Date'] = pd.to_datetime(product_data['Order Date'], errors='coerce')

invalid_dates = product_data[product_data['Order Date'].isnull()]
print("Invalid dates in Product data:")
print(invalid_dates)

out_of_range_stock = product_data[product_data['StockLevel'] < 0]
print("Products with negative stock levels:")
print(out_of_range_stock)


Invalid dates in Product data:
                            ProductName                        Description  \
65                   COM-G3-L1-403-S5S5                A Four-way combiner   
89       TP-Link DOCSIS 3.0 Cable Modem                High-Speed Internet   
463  DOCSIS 3.1 Cable Modem (CM1000) -4  A Nighthawk Multi-Gig Cable Modem   

          Product Image Product Cateogy Name Model Number  \
65   No Image Available             Combiner       BGH684   
89            modem.jpg                Modem       NLX300   
463  No Image Available                Modem     CAR-5370   

                            Serial Number  StockLevel  ReorderPoint  \
65   dad627fe-e8f6-49ec-a798-9a7460577397         300            75   
89   75c083e7-f683-41e5-be69-379f89f5aa1b         200            25   
463  330668d4-072b-488f-91a0-8b13937e3891         300           100   

    Supplier Name        Supplier Mail  Supplier  Contact Order Date  \
65        Synergy      synerg@mail.com         7865434

In [15]:
user_data.columns = user_data.columns.str.replace('\xa0', ' ', regex=True)

user_data['Phone'] = user_data['Phone'].astype(str)
user_data['Email'] = user_data['Email'].astype(str)

user_data['Valid Phone'] = user_data['Phone'].str.isdigit() & (user_data['Phone'].str.len() == 10)

user_data['Valid Email'] = user_data['Email'].fillna('').str.match(r'^[\w\.-]+@[\w\.-]+\.\w+$')

print("Invalid phone numbers:")
print(user_data[~user_data['Valid Phone']])

print("Invalid emails:")
print(user_data[~user_data['Valid Email']])


Invalid phone numbers:
Empty DataFrame
Columns: [UserId, UserName, Password, First Name, Last Name, Type, Phone, Email, Valid Phone, Valid Email]
Index: []
Invalid emails:
Empty DataFrame
Columns: [UserId, UserName, Password, First Name, Last Name, Type, Phone, Email, Valid Phone, Valid Email]
Index: []


In [21]:
product_data.to_excel('Cleaned_Product_Data.xlsx', index=False)
user_data.to_excel('Cleaned_User_Data.xlsx', index=False)

print("Cleaned data saved successfully!")



Cleaned data saved successfully!


In [23]:
missing_values = product_data.isnull().sum()
print("Missing values in Product Data:")
print(missing_values)


Missing values in Product Data:
ProductName             0
Description             0
Product Image           0
Product Cateogy Name    0
Model Number            0
Serial Number           0
StockLevel              0
ReorderPoint            0
Supplier Name           0
Supplier Mail           0
Supplier  Contact       0
Order Date              3
Quantity                0
Order Status            0
dtype: int64


In [25]:
# Fill missing 'Product Image' with a placeholder
product_data['Product Image'] = product_data['Product Image'].fillna('No Image Available')

# Optionally drop rows with missing critical information
product_data = product_data.dropna(subset=['ProductName', 'Supplier Name'])

print("Data after handling missing values:")
print(product_data.isnull().sum())


Data after handling missing values:
ProductName             0
Description             0
Product Image           0
Product Cateogy Name    0
Model Number            0
Serial Number           0
StockLevel              0
ReorderPoint            0
Supplier Name           0
Supplier Mail           0
Supplier  Contact       0
Order Date              3
Quantity                0
Order Status            0
dtype: int64


In [27]:
# Drop duplicate rows if any
product_data = product_data.drop_duplicates()

print(f"Data shape after removing duplicates: {product_data.shape}")


Data shape after removing duplicates: (500, 14)


In [29]:
# Convert 'Order Date' to datetime
product_data['Order Date'] = pd.to_datetime(product_data['Order Date'], errors='coerce')

# Check for invalid dates
invalid_dates = product_data[product_data['Order Date'].isnull()]
print("Rows with invalid dates:")
print(invalid_dates)

# Remove or fill rows with invalid dates
product_data = product_data.dropna(subset=['Order Date'])

# Create new columns for Year, Month, Day, and other date-related information
product_data['Year'] = product_data['Order Date'].dt.year
product_data['Month'] = product_data['Order Date'].dt.month
product_data['Day'] = product_data['Order Date'].dt.day
product_data['Day Name'] = product_data['Order Date'].dt.day_name()  # e.g., Monday
product_data['Month Name'] = product_data['Order Date'].dt.month_name()  # e.g., January

# Display a preview of the updated dataframe
print(product_data[['Order Date', 'Year', 'Month', 'Day', 'Day Name', 'Month Name']].head())


Rows with invalid dates:
                            ProductName                        Description  \
65                   COM-G3-L1-403-S5S5                A Four-way combiner   
89       TP-Link DOCSIS 3.0 Cable Modem                High-Speed Internet   
463  DOCSIS 3.1 Cable Modem (CM1000) -4  A Nighthawk Multi-Gig Cable Modem   

          Product Image Product Cateogy Name Model Number  \
65   No Image Available             Combiner       BGH684   
89            modem.jpg                Modem       NLX300   
463  No Image Available                Modem     CAR-5370   

                            Serial Number  StockLevel  ReorderPoint  \
65   dad627fe-e8f6-49ec-a798-9a7460577397         300            75   
89   75c083e7-f683-41e5-be69-379f89f5aa1b         200            25   
463  330668d4-072b-488f-91a0-8b13937e3891         300           100   

    Supplier Name        Supplier Mail  Supplier  Contact Order Date  \
65        Synergy      synerg@mail.com         7865434961   

In [31]:
# Standardize text in 'Product Cateogy Name' and 'Order Status'
product_data['Product Cateogy Name'] = product_data['Product Cateogy Name'].str.title()
product_data['Order Status'] = product_data['Order Status'].str.capitalize()


In [33]:
# Identify outliers or invalid values
out_of_range_stock = product_data[product_data['StockLevel'] < 0]
print("Outliers in Stock Level:")
print(out_of_range_stock)

# Remove rows with invalid stock levels
product_data = product_data[product_data['StockLevel'] >= 0]


Outliers in Stock Level:
Empty DataFrame
Columns: [ProductName, Description, Product Image, Product Cateogy Name, Model Number, Serial Number, StockLevel, ReorderPoint, Supplier Name, Supplier Mail, Supplier  Contact, Order Date, Quantity, Order Status, Year, Month, Day, Day Name, Month Name]
Index: []


In [35]:
# Validate Supplier Email format
product_data['Valid Supplier Email'] = product_data['Supplier Mail'].str.contains(r'^[\w\.-]+@[\w\.-]+\.\w+$')

# Find invalid emails
invalid_emails = product_data[~product_data['Valid Supplier Email']]
print("Invalid supplier emails:")
print(invalid_emails)


Invalid supplier emails:
Empty DataFrame
Columns: [ProductName, Description, Product Image, Product Cateogy Name, Model Number, Serial Number, StockLevel, ReorderPoint, Supplier Name, Supplier Mail, Supplier  Contact, Order Date, Quantity, Order Status, Year, Month, Day, Day Name, Month Name, Valid Supplier Email]
Index: []


In [37]:
# Save the cleaned data
product_data.to_excel('Cleaned_Product_Data.xlsx', index=False)
print("Cleaned Product Data saved successfully!")


Cleaned Product Data saved successfully!
